In [55]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import geopandas as gpd

## Data général

In [56]:
df_arrivals = pd.read_csv('../data/df_arrivals.csv')
df_departures = pd.read_csv('../data/df_departures.csv')
df_low_cost = pd.read_csv('../data/Low_Cost_Airlines.csv')
df_international_airports = pd.read_csv("../data/List_of_international_airports_by_country.csv")

In [57]:
def plot_nan_percentage(df):
    _, stats_df = get_nan_rows(df)
    return stats_df

def get_nan_rows(df):
    null_df = [df[df[column].isna()] for column in df.columns]
    stats_df = [(column, df[column].isna().sum(), (df[column].isna().sum() / df.shape[0]) * 100) for column in df.columns]
    stats_df = pd.DataFrame(stats_df, columns=["column", "nan_count", "nan_percentage"])
    return null_df, stats_df

In [58]:
stats_df_arr = plot_nan_percentage(df_arrivals)
stats_df_dep = plot_nan_percentage(df_departures)

df_arrivals_cleaned = df_arrivals[pd.notna(df_arrivals["Airline Name"])]
df_departures_cleaned = df_departures[pd.notna(df_departures["Airline Name"])]
df_arrivals_cleaned.shape, df_departures_cleaned.shape, df_low_cost.shape

columns_to_drop = stats_df_arr[stats_df_arr["nan_percentage"] > 60]["column"].values
df_arrivals_cleaned = df_arrivals_cleaned.drop(columns=columns_to_drop)
columns_to_drop = stats_df_dep[stats_df_dep["nan_percentage"] > 65]["column"].values
df_departures_cleaned = df_departures_cleaned.drop(columns=columns_to_drop)

In [59]:
def separate_df(df):
    df_low_cost_service = df[df["Airline Name"].isin(df_low_cost["Low Cost Airlines"])]
    df_full_service = df[~df["Airline Name"].isin(df_low_cost["Low Cost Airlines"])]
    return df_low_cost_service, df_full_service

df_arrivals_low_cost, df_arrivals_full_service = separate_df(df_arrivals_cleaned)
df_departures_low_cost, df_departures_full_service = separate_df(df_departures_cleaned)

In [60]:
def join_dataset(df_arrivals, df_departures):
    df_matched = df_arrivals.join(df_departures, how='inner', lsuffix='_arr', rsuffix='_dep')
    columns_to_drop = [col for col in df_matched.columns if col.endswith('_dep') and col[:-4] + '_arr' in df_matched.columns]
    df_matched.drop(columns=columns_to_drop, inplace=True)
    df_matched.rename(columns=lambda x: x.replace('_arr', ''), inplace=True)
    return df_matched

df_matched_low_cost = join_dataset(df_arrivals_low_cost, df_departures_low_cost)
df_matched_full_service = join_dataset(df_arrivals_full_service, df_departures_full_service)

In [61]:
aircraft_low_cost = df_matched_low_cost.groupby("Aircraft Code").apply(len).sort_values(ascending=False)
aircraft_full_service = df_matched_full_service.groupby("Aircraft Code").apply(len).sort_values(ascending=False)
all_aircraf_code = list(set(df_arrivals["Aircraft Code"].unique()).union(df_departures["Aircraft Code"].unique()))
aircraft_low_cost_complete = aircraft_low_cost.reindex(all_aircraf_code, fill_value=0)
aircraft_full_service_complete = aircraft_full_service.reindex(all_aircraf_code, fill_value=0)
combined_aircraft_usage = pd.concat([aircraft_low_cost_complete, aircraft_full_service_complete], axis=1, keys=['Low Cost', 'Full Service'])

In [62]:
companies_low_cost_freq = df_matched_low_cost.groupby("Airline Name").apply(len).sort_values(ascending=False)
companies_full_service_freq = df_matched_full_service.groupby("Airline Name").apply(len).sort_values(ascending=False)

companies_low_cost_freq = companies_low_cost_freq.to_frame(name='Frequency')
companies_full_service_freq = companies_full_service_freq.to_frame(name='Frequency')

companies_low_cost_freq['Airline Service'] = 'Low-Cost'
companies_full_service_freq['Airline Service'] = 'Full-Service'

all_comp_freq = pd.concat([companies_low_cost_freq, companies_full_service_freq], axis=0)
all_comp_freq = all_comp_freq.reset_index()

## Data USA

In [63]:
def is_low_cost(airline):
    for low_cost in low_cost_airlines["Low Cost Airlines"].values:
        if low_cost in airline:
            return True
    return False

def month_to_season(month):
    if month in ['December', 'January', 'February']:
        return 'Winter'
    elif month in ['March', 'April', 'May']:
        return 'Spring'
    elif month in ['June', 'July', 'August']:
        return 'Summer'
    elif month in ['September', 'October', 'November']:
        return 'Fall'
    
    
def get_stats(group):
    return {'count': group.count(),'min': group.min(), 'max': group.max(), 'mean': group.mean()}

df_flights_cleaned = pd.read_csv("../data/flights_cleaned.csv",low_memory=False)
df_flights_cleaned['DATE'] = pd.to_datetime(df_flights_cleaned['DATE'])
df_flights_cleaned['MONTH'] = df_flights_cleaned['DATE'].dt.month_name()
df_flights_cleaned['SEASON'] = df_flights_cleaned['MONTH'].apply(month_to_season)

all_usa_airlines = pd.read_csv("../analysis/data/USA/airlines.csv")
all_usa_airports = pd.read_csv("../analysis/data/USA/airports.csv")
low_cost_airlines = pd.read_csv("../data/Low_Cost_Airlines.csv")

try:
    print("Reading from file")
    df_flights_cleaned_airlines = pd.read_csv("../analysis/data/flights_cleaned_airlines.csv",low_memory=False)
except:
    print("Creating new file")
    non_used_columns = ['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK','FLIGHT_NUMBER', 'TAIL_NUMBER','TAXI_OUT',
        'WHEELS_OFF','WHEELS_ON', 'TAXI_IN','AIR_TIME','DIVERTED', 'CANCELLED', 'CANCELLATION_REASON','AIR_SYSTEM_DELAY', 'SECURITY_DELAY',
        'AIRLINE_DELAY','LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY']
    df_flights_cleaned = df_flights_cleaned.drop(non_used_columns,axis=1)

    us_low_cost = all_usa_airlines[all_usa_airlines["AIRLINE"].apply(is_low_cost)]
    us_full_serivce = all_usa_airlines[~all_usa_airlines["AIRLINE"].apply(is_low_cost)]

    df_flights_cleaned_airlines = df_flights_cleaned.copy()
    df_flights_cleaned_airlines.insert(loc = 0, column = "AIRLINE_NAME", value = df_flights_cleaned["AIRLINE"].map(all_usa_airlines.set_index("IATA_CODE")["AIRLINE"]))
    df_flights_cleaned_airlines.insert(loc = 1, column = "AIRLINE_SERVICE", value = df_flights_cleaned_airlines["AIRLINE_NAME"].apply(lambda x: "Low Cost" if x in us_low_cost["AIRLINE"].values else "Full Service"))

    df_flights_cleaned_airlines.to_csv("../analysis/data/flights_cleaned_airlines.csv", index=False)

def analyse_by_month(month_number=4):
    df_month = df_flights_cleaned[df_flights_cleaned['MONTH'] == month_number]
    return df_month

C:\Users\halim\AppData\Local\Temp\ipykernel_18376\979859748.py:80: FutureWarning:

The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.



Reading from file


In [64]:
def get_stats(group):
    return {'count': group.count(),'min': group.min(), 'max': group.max(), 'mean': group.mean()}

delay_per_airline_stats = df_flights_cleaned['DEPARTURE_DELAY'].groupby(df_flights_cleaned['AIRLINE']).apply(get_stats).unstack()
all_usa_airlines = pd.read_csv("../analysis/data/USA/airlines.csv")
us_low_cost = all_usa_airlines[all_usa_airlines["AIRLINE"].apply(is_low_cost)]
us_full_service = all_usa_airlines[~all_usa_airlines["AIRLINE"].apply(is_low_cost)]

In [65]:
delay_per_airline_stats.reset_index(inplace=True)

In [66]:
# Creating mappings for service type and airline name
service_map = pd.concat([
    us_low_cost.set_index('IATA_CODE')['AIRLINE'].apply(lambda x: 'Low-Cost'),
    us_full_service.set_index('IATA_CODE')['AIRLINE'].apply(lambda x: 'Full-Service')
])

name_map = pd.concat([
    us_low_cost.set_index('IATA_CODE')['AIRLINE'],
    us_full_service.set_index('IATA_CODE')['AIRLINE']
])

# If delay_per_airline_stats uses IATA codes as index:
delay_per_airline_stats['Service Type'] = delay_per_airline_stats.index.map(service_map)
delay_per_airline_stats['Airline Name'] = delay_per_airline_stats.index.map(name_map)

# If the index is not IATA codes, adjust accordingly
# Example assuming 'Airline Code' is a column in delay_per_airline_stats:
delay_per_airline_stats['Service Type'] = delay_per_airline_stats['AIRLINE'].map(service_map)
delay_per_airline_stats['Airline Name'] = delay_per_airline_stats['AIRLINE'].map(name_map)

## Styles

In [67]:
primary_color = '#5647FF'
secondary_color = '#FF4754'
terciary_color = '#1B1C22'
base_color = '#FFFFFF'
black_color = '#000000'
bonus_1 = '#FF7700'
bonus_2 = "#00AAFF"
height="300vh"

style_h1 = {
    'textAlign': 'center',
    'color': f'{base_color}',
    'padding': '10px',
    'fontFamily': 'Verdana',
    'fontWeight': 'bold',
    'marginBottom': '10px'
}
style_h3 = {
    'textAlign': 'left',
    'color': f'{terciary_color}',
    'padding': '5px',
    'fontFamily': 'Verdana',
}

style_tab = {
    'border':'none', 
    'backgroundColor': 
    f'{primary_color}', 
    'color': f'{base_color}', 
    'padding': '20px', 
    'marginLeft': '10px',
    'fontFamily': 'Verdana',
    'width': '100%'
}
style_tab_selected = {
    'border':'none', 
    'backgroundColor': f'{base_color}', 
    'color': f'{primary_color}', 
    'borderTopLeftRadius': '20px', 
    'borderBottomLeftRadius': '20px',
    'padding': '20px', 
    'margin': '3px', 
    'fontFamily': 'Verdana',
    'width': '110%',
    'fontWeight': 'bold'
}

style_main_layout = {
    'display': 'flex',
    'flex-direction': 'row',  
    'height': f'{height}',
    'backgroundColor': f'{base_color}',
    'borderRadius': '30px',
}

style_sidebar = {
    'display': 'flex',
    'flex-direction': 'column',  
    'background-color': f'{primary_color}',
    'width': '300px',
    'color': 'white',
    'padding': '20px',
    'borderTopLeftRadius': '20px',
    'borderBottomLeftRadius': '20px',
}

style_tabs = {
    'flex-direction': 'column',  # Aligns tabs vertically
    'height': f'{height}',
}

style_content = {
    'flex': '1',
    'padding': '20px'
}

category_colors = {
    'Both': primary_color,
    'Full-Service': secondary_color,
    'Low-Cost': bonus_2,
    'None': base_color  # Assuming 'None' is a category for no service
}

## Layouts

In [68]:
lda_html = open("lda.html", "r").read()
lda_html_neg = open("lda_neg.html", "r").read()

layout_client = html.Div([
    html.H3("Visualisation des sujets", style=style_h3),
    dcc.Dropdown(
        id='airline-type-dropdown-tab1',
        options=[
            {'label': 'Avis positif', 'value': 'POS'},
            {'label': 'Avis negatif', 'value': 'NEG'},
        ],
        value='AL',
        clearable=False,
        style={'fontFamily': 'Verdana'}
    ),
    html.Iframe(
        id='lda-iframe',
        style={"width": "100%", "height": "800px"}
    )
])

In [69]:
layout_ls_vs_fs =  html.Div([
    html.H3('Analyse par type de companie', style=style_h3),
    dcc.Dropdown(
        id='airline-type-dropdown',
        options=[
            {'label': 'Toute', 'value': 'AL'},
            {'label': 'Low Cost', 'value': 'LC'},
            {'label': 'Full Service', 'value': 'FS'}
        ],
        value='AL',
        clearable=False,
        style={'fontFamily': 'Verdana'}
    ),
    html.Div([
        dcc.Graph(id='aircraft-freq-graph', style={'width':'50%', 'flex':'1'}),
        dcc.Graph(id='airline-type-graph',style={'display':'none', 'flex':'1'}),
    ], style={'display':'flex', 'flex-direction':'row'}),

    html.Div([
        dcc.Graph(id='airline-freq', style={'width':'50%', 'flex':'1'}),
        dcc.Graph(id='world-map-graph', style={'width':'50%', 'flex':'1'}),
    ], style={'display':'flex', 'flex-direction':'row'}),

], style={'width':'97%'})

In [70]:
layout_usa =  html.Div([
    html.H3('Analyse sur les USA', style=style_h3),
    dcc.Dropdown(
        id='airline-type-dropdown-tab3',
        options=[
            {'label': 'Par Jours', 'value': 'DA'},
            {'label': 'Par Mois', 'value': 'MO'},
            {'label': 'Par Saison', 'value': 'SA'}
        ],
        value='DA',
        clearable=False,
        style={'fontFamily': 'Verdana'}
    ),
    dcc.Graph(id='aircraft-delay-graph-tab3', style={'width':'100%', 'flex':'1'}),
    html.Div([
        html.H4('Trier les valeurs', style=style_h3),
        dcc.Dropdown(
            id='airline-delay-dropdown-sort',
            options=[
                {'label': 'Non', 'value': 'NO'},
                {'label': 'Oui', 'value': 'YES'},
            ],
            value='NO',
            clearable=False,
            style={'fontFamily': 'Verdana','width':'30%'}
        ),
        dcc.Graph(id='airline-delay-graph-tab3', style={'width':'100%', 'flex':'1'}),
    ])
])

## Dashboard

In [71]:
from dash import Dash, dcc, html
from dash.dependencies import Input, Output

app = Dash(__name__,suppress_callback_exceptions=True)

app.layout = html.Div([
    html.Div([
        html.H1('Dashboard', style=style_h1),
        html.Hr(style={'border': '1px solid white','width': '100%', 'marginBottom': '70px'}),
        dcc.Tabs(id="tabs", value='tab-2', children=[
            dcc.Tab(label="Analyse sur les avis clients", value='tab-1', style=style_tab, selected_style=style_tab_selected),
            dcc.Tab(label="Analyse type de service", value='tab-2', style=style_tab, selected_style=style_tab_selected),
            dcc.Tab(label="Analyse sur les États Unis", value='tab-3', style=style_tab, selected_style=style_tab_selected),
        ], style=style_tabs),
    ], style=style_sidebar),
    
    html.Div(id='tabs-content', style=style_content)
], style=style_main_layout)

@app.callback(Output('tabs-content', 'children'),
              Input('tabs', 'value'))
def render_content(tab):
    if tab == 'tab-1':
        return layout_client
    elif tab == 'tab-2':
        return layout_ls_vs_fs
    elif tab == 'tab-3':
        return layout_usa
    

## Functions

In [72]:
@app.callback(
    Output('lda-iframe', 'srcDoc'),
    Input('airline-type-dropdown-tab1', 'value')
)
def get_post_type(selected_type):
    fichier = lda_html
    if selected_type == "NEG":
        fichier = lda_html_neg
    return fichier

@app.callback(
    Output('airline-delay-graph-tab3', 'figure'),
    Input('airline-delay-dropdown-sort', 'value'),
)
def generate_figure(selected_sort='AL'):

    delay_per_airline_stats['Retard en minutes'] = delay_per_airline_stats['mean']
    display_df = delay_per_airline_stats.copy()
    if selected_sort == 'YES':
        display_df = delay_per_airline_stats.sort_values(by='mean',ascending=False).copy()
   
    fig_airlines_delay = px.bar(
        display_df,
        x='Airline Name',              # Ensure this column has unique airline names
        y='Retard en minutes',
        color='Service Type',          # This will color by service type but not group by it
        title="Retard moyen par compagnie aérienne",
        labels={
            'Airline Name': 'Compagnie Aérienne',
            'Retard en minutes': 'Retard en minutes'
        },
        category_orders={"Airline Name": display_df['Airline Name'].tolist()},  # Ensures the sort order of x-axis
        barmode='group'  # Important to ensure no physical grouping
    )

    fig_airlines_delay.update_layout(
        xaxis_title="Compagnie Aérienne",
        yaxis_title="Retard en minutes",
        legend_title="Type de Service"
)
    return fig_airlines_delay    

In [73]:
import geopandas as gpd

def preprocess_usa(country):
    if country == "United States":
        return "United States of America"
    return country



@app.callback(
    [Output('airline-type-graph', 'figure'),
     Output('aircraft-freq-graph', 'figure'),
     Output('world-map-graph', 'figure'),
        Output('airline-freq', 'figure')
    ],
    Input('airline-type-dropdown', 'value')
)
def update_graphs(selected_type):
    fig_airline_type = None
    if selected_type == 'AL':
        fig_airline_type = px.box(
            df_flights_cleaned_airlines.sample(100), 
            y="DISTANCE", 
            color="AIRLINE_SERVICE",
            labels={
                "DISTANCE": "Distance (miles)",
                "AIRLINE_SERVICE": "Airline Service"
            },
            title="Distance des vols par type de companie"
        )

        fig_airline_type.update_layout(
            xaxis_title="Airline Service",
            yaxis_title="Distance (miles)",
            legend_title="Airline Service"
        )

        # fig_airline_type = px.bar(
        #     x=["Low Cost", "Full Service"],
        #     y=[df_matched_low_cost.shape[0],df_matched_full_service.shape[0]],
        #     title="Nombre de vols par type de companie",
        #     labels={'x': 'Type de companie', 'y': ''},
        #     color=[primary_color, secondary_color],
        # )
    else:
        fig_airline_type = {}

    company_type = 'Low Cost'
    if selected_type == 'FS':
        company_type = 'Full Service'

    
    if 'index' in combined_aircraft_usage.columns:
        top_aircraft = combined_aircraft_usage.sort_values(by=company_type, ascending=False).head(10)
    else:
        top_aircraft = combined_aircraft_usage.reset_index().sort_values(by=company_type, ascending=False).head(10)
        
    if selected_type == 'LC':
        top_aircraft = top_aircraft[["Aircraft Code", "Low Cost"]]
        vars = ['Low Cost']
    elif selected_type == 'FS':
        top_aircraft = top_aircraft[["Aircraft Code", "Full Service"]]
        vars = ['Full Service']
    else:
        vars = ['Low Cost', 'Full Service']

    top_aircraft_long = top_aircraft.reset_index().melt(id_vars='Aircraft Code', value_vars=vars, var_name='Type', value_name='Number of Flights')
    # Create the bar plot with Plotly Express
    fig_aircraft_freq = px.bar(
        top_aircraft_long,
        x='Aircraft Code',
        y='Number of Flights',
        color='Type',
        title="Utilisation des avions par type de companie",
        labels={'Aircraft Code': 'Aircraft Code', 'Number of Flights': 'Number of Flights'},
        barmode='group'
    )

    # Load world map
    world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

    # Assumons que df_departures_full_service et df_departures_low_cost sont déjà définis
    countries_full_service = df_departures_full_service["Destination Country"].apply(preprocess_usa).unique()
    countries_low_cost = df_departures_low_cost["Destination Country"].apply(preprocess_usa).unique()

    # Créer une nouvelle colonne pour représenter le service aérien
    world['airline_service'] = 'None'
    if selected_type == 'FS' or selected_type == 'AL':
        world.loc[world['name'].isin(countries_full_service), 'airline_service'] = 'Full-Service'
    if selected_type == 'LC' or selected_type == 'AL':
        world.loc[world['name'].isin(countries_low_cost), 'airline_service'] = 'Low-Cost'
    if selected_type == 'AL':
        world.loc[world['name'].isin(set(countries_full_service).intersection(countries_low_cost)), 'airline_service'] = 'Both'
    
    # Convertir GeoDataFrame en JSON pour Plotly
    world_json = world.__geo_interface__
    fig_world = px.choropleth(
        world,
        geojson=world_json,
        locations=world.index,
        color=world['airline_service'],
        color_discrete_map=category_colors,  # Using a discrete color map
        projection='equirectangular',
        title='Service Aérien par Pays',
        labels={'airline_service': 'Type de Service Aérien'}
    )

    fig_world.update_geos(fitbounds="locations")
    all_comp_freq = pd.concat([companies_low_cost_freq, companies_full_service_freq], axis=0)
    all_comp_freq = all_comp_freq.reset_index()

    if selected_type == 'LC':
        filtered_data = all_comp_freq[all_comp_freq['Airline Service'] == 'Low-Cost']
    elif selected_type == 'FS':
        filtered_data = all_comp_freq[all_comp_freq['Airline Service'] == 'Full-Service']
    else:
        filtered_data = all_comp_freq  # Or handle differently if needed

    # Sorting and selecting top 10
    top_airlines_freq = filtered_data.sort_values(by='Frequency', ascending=False).head(10)

    # Creating the bar chart
    fig_airlines_freq = px.bar(
        top_airlines_freq,
        x='Airline Name',
        y='Frequency',
        color='Airline Service',  # Using the service type for coloring
        title="Fréquence des vols par compagnie aérienne",
        labels={'Airline Name': 'Airline Name', 'Frequency': 'Number of Flights'},
        barmode='group'
    )

    return fig_airline_type, fig_aircraft_freq, fig_world, fig_airlines_freq

@app.callback(
    [
        Output('airline-type-graph', 'style'),
        Output('aircraft-freq-graph', 'style')
    ],
    Input('airline-type-dropdown', 'value')
)
def toggle_visibility(selected_type):
    if selected_type == 'AL':
        return {'display': 'flex', 'width':'50%'}, {'display': 'flex','width':'50%'}
    else:
        return {'display': 'none'}, {'width':'100%'}
    

@app.callback(
    Output('aircraft-delay-graph-tab3', 'figure'),
    Input('airline-type-dropdown-tab3', 'value'),
    allow_duplicate=True
)
def update_graphs_tab3(selected_type):   
    temp_df = df_flights_cleaned.sample(df_arrivals_cleaned.shape[0]//2)
    if selected_type == 'DA':
        # temp_df = df_flights_cleaned.sample(1000)
        temp_df['DATE'] = pd.to_datetime(temp_df['DATE'])
        temp_df['WEEKDAY'] = temp_df['DATE'].dt.day_name()
        average_delay_weekday = temp_df.groupby('WEEKDAY')['DEPARTURE_DELAY'].mean()

        fig = px.bar(
            average_delay_weekday,
            title='Retard moyen par jours',
            labels={'index': 'Weekday', 'value': 'Retard moyen (minutes)'}
        )
    elif selected_type == 'MO':
        # temp_df = df_flights_cleaned.sample(1000)
        temp_df['MONTH'] = temp_df['DATE'].dt.month_name()
        average_delay_month = temp_df.groupby('MONTH')['DEPARTURE_DELAY'].mean()

        fig = px.bar(
            average_delay_month,
            title='Retard moyen par mois',
            labels={'index': 'Month', 'value': 'Retard moyen (minutes)'}
        )
    elif selected_type == 'SA':
        average_delay_season = temp_df.groupby('SEASON')['DEPARTURE_DELAY'].mean()
        fig = px.bar(
            average_delay_season.reindex(['Winter', 'Spring', 'Summer', 'Fall']),
            title='Retard moyen par saison',
            labels={'index': 'Season', 'value': 'Retard moyen (minutes)'},
        )
    return fig

In [74]:
app.run_server(debug=True,use_reloader=False)

C:\Users\halim\AppData\Local\Temp\ipykernel_18376\979859748.py:80: FutureWarning:

The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.

